## This Notebook is for running the LibInvent Module of the reinvent4 Repo
## Author - Andrew Tom CSE23269
note: Please run in the reinvent4 environment. 

# After Making sure that the sampling.toml line is setup for using the libinvent mode run the below commands. 


*******************
Scaffolds in scaffolds.smi

[*:1]c1c[nH]nc1Nc1cc([*:2])c(C[*:3])cc1

[*:1]C1=C2C(=NC=N1)C(C[*:2])NO2

*****************


In [16]:
import sys
!{sys.executable} -m reinvent sampling.toml

16:44:19 <INFO> Started REINVENT 4.5.11 (C) AstraZeneca 2017, 2023 on 2025-06-23
16:44:19 <INFO> Command line: C:\Users\andre\anaconda3\envs\reinvent4\lib\site-packages\reinvent\__main__.py sampling.toml
16:44:19 <INFO> Reading run configuration from C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\sampling.toml using format toml
16:44:19 <INFO> User andre on host Andrews-Legion
16:44:19 <INFO> Python version 3.10.18
16:44:19 <INFO> PyTorch version 2.6.0+cu124, git 2236df1770800ffea5697b11b0bb0d910b2e59e1
16:44:19 <INFO> PyTorch compiled with CUDA version 12.4
16:44:19 <INFO> RDKit version 2025.03.3
16:44:19 <INFO> Platform Windows-10-10.0.26100-SP0
16:44:19 <INFO> Number of PyTorch CUDA devices 1
16:44:19 <INFO> Using CUDA device:0 NVIDIA GeForce RTX 4060 Laptop GPU
16:44:19 <INFO> GPU memory: 7099 MiB free, 8187 MiB total
16:44:19 <INFO> Writing JSON config file to C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\_sampling.json
16:44:19 <INFO> Starting Sampling
16:44:19 <INFO> C:\U

results of sampling is stored in sampling.csv

In [17]:
import sys
!{sys.executable} -m reinvent my_scoring.toml

16:46:26 <INFO> Started REINVENT 4.5.11 (C) AstraZeneca 2017, 2023 on 2025-06-23
16:46:26 <INFO> Command line: C:\Users\andre\anaconda3\envs\reinvent4\lib\site-packages\reinvent\__main__.py my_scoring.toml
16:46:26 <INFO> Reading run configuration from C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\my_scoring.toml using format toml
16:46:26 <INFO> User andre on host Andrews-Legion
16:46:26 <INFO> Python version 3.10.18
16:46:26 <INFO> PyTorch version 2.6.0+cu124, git 2236df1770800ffea5697b11b0bb0d910b2e59e1
16:46:26 <INFO> PyTorch compiled with CUDA version 12.4
16:46:26 <INFO> RDKit version 2025.03.3
16:46:26 <INFO> Platform Windows-10-10.0.26100-SP0
16:46:26 <INFO> Number of PyTorch CUDA devices 1
16:46:26 <INFO> Using CUDA device:0 NVIDIA GeForce RTX 4060 Laptop GPU
16:46:26 <INFO> GPU memory: 7099 MiB free, 8187 MiB total
16:46:26 <INFO> Writing JSON config file to C:\Users\andre\OneDrive\Desktop\REINVENT4\configs\scoring_simple.json
16:46:26 <INFO> Scoring SMILES from file C:\U

results of scoring are stored in scoring_results.csv


In [18]:
import pandas as pd

# Load dataset
df = pd.read_csv('scoring_results.csv')  # <-- use your actual file name

# Start with a copy
filtered = df.copy()

# Only apply Tanimoto filter if the column exists in the file
if 'Tanimoto' in filtered.columns:
    filtered = filtered[filtered['Tanimoto'] >= 0.6]

# Remove Input_SMILES column if it exists
if 'Input_SMILES' in filtered.columns:
    filtered = filtered.drop(columns=['Input_SMILES'])

# Apply property filters (independent of Tanimoto)
filtered = filtered[
    (filtered['Molecular weight'] <= 550) &
    (filtered['QED'] >= 0.75) &
    (filtered['SA score'] <= 4) &
    (filtered['SlogP (RDKit)'] >= -1) &
    (filtered['SlogP (RDKit)'] <= 5)
]

print(f"✅ Filtered {len(filtered)} molecules out of {len(df)} total.")

# Save output
filtered.to_csv('filtered_scoring_results.csv', index=False)



✅ Filtered 1811 molecules out of 20688 total.


## For Displaying the structures

In [19]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import Image
import numpy as np

# Load molecules
df = pd.read_csv("filtered_scoring_results.csv")
smiles_list = df["SMILES"].tolist()
mols = [Chem.MolFromSmiles(smi) for smi in smiles_list if Chem.MolFromSmiles(smi)]

# Split into chunks of N molecules per page
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

pages = list(chunks(mols, 20))  # 20 mols per page

images = []
for chunk in pages:
    img_array = Draw.MolsToGridImage(
        chunk,
        molsPerRow=4,
        subImgSize=(300, 300),
        useSVG=False,
        returnPNG=False  # Important: this returns a numpy array
    )
    # Convert RDKit image (PIL or array) to actual PIL Image
    pil_img = img_array if isinstance(img_array, Image.Image) else Image.fromarray(np.array(img_array))
    images.append(pil_img.convert("RGB"))

# Save to PDF
if images:
    images[0].save("molecule_grid.pdf", save_all=True, append_images=images[1:])
